In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import RobustScaler
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

In [2]:
data=pd.read_csv('final_dataset.csv')

data

,year,İBBS3,city,population,total_cases,carried_over_from,opened_inside_year,finished,carried_over_to,returned,area
0,2021,TR100,İstanbul,15099946.0,2378387,1536743,841644,836525,1541862,12697,5461
1,2021,TR211,Tekirdağ,1105759.0,127163,59444,67719,62603,64560,516,6190
2,2021,TR212,Edirne,406215.0,50363,24888,25475,27064,23299,457,6145
3,2021,TR213,Kırklareli,363245.0,31716,12732,18984,18939,12777,215,6459
4,2021,TR221,Balıkesir,1238619.0,139589,66047,73542,74267,65322,913,14583
...,...,...,...,...,...,...,...,...,...,...,...
1048,2009,TRC22,Diyarbakır,1514793.0,89211,47222,41989,38964,50247,602,15101
1049,2009,TRC31,Mardin,736455.0,24841,7067,17774,15843,8998,390,8780
1050,2009,TRC32,Batman,497841.0,22390,8826,13564,13984,8406,189,4477
1051,2009,TRC33,Şırnak,430006.0,13615,3692,9923,9707,3908,47,7078


In [3]:
# İBBS3, total_cases and returned columns are not useful

data = data.drop(columns=['İBBS3', 'total_cases', 'returned'])
data


,year,city,population,carried_over_from,opened_inside_year,finished,carried_over_to,area
0,2021,İstanbul,15099946.0,1536743,841644,836525,1541862,5461
1,2021,Tekirdağ,1105759.0,59444,67719,62603,64560,6190
2,2021,Edirne,406215.0,24888,25475,27064,23299,6145
3,2021,Kırklareli,363245.0,12732,18984,18939,12777,6459
4,2021,Balıkesir,1238619.0,66047,73542,74267,65322,14583
...,...,...,...,...,...,...,...,...
1048,2009,Diyarbakır,1514793.0,47222,41989,38964,50247,15101
1049,2009,Mardin,736455.0,7067,17774,15843,8998,8780
1050,2009,Batman,497841.0,8826,13564,13984,8406,4477
1051,2009,Şırnak,430006.0,3692,9923,9707,3908,7078


In [4]:
# Transform the year column to time_since_event in order for machine to understand the year concept
current_year = 2024
data['time_since_event'] = current_year - data['year']

# Drop the 'year' column
data.drop('year', axis=1, inplace=True)

# Add the population density column
data['population_density'] = data['population'] / data['area']

data.drop(columns=['area'])


data

,city,population,carried_over_from,opened_inside_year,finished,carried_over_to,area,time_since_event,population_density
0,İstanbul,15099946.0,1536743,841644,836525,1541862,5461,3,2765.051456
1,Tekirdağ,1105759.0,59444,67719,62603,64560,6190,3,178.636349
2,Edirne,406215.0,24888,25475,27064,23299,6145,3,66.104963
3,Kırklareli,363245.0,12732,18984,18939,12777,6459,3,56.238582
4,Balıkesir,1238619.0,66047,73542,74267,65322,14583,3,84.935816
...,...,...,...,...,...,...,...,...,...
1048,Diyarbakır,1514793.0,47222,41989,38964,50247,15101,15,100.310774
1049,Mardin,736455.0,7067,17774,15843,8998,8780,15,83.878702
1050,Batman,497841.0,8826,13564,13984,8406,4477,15,111.199687
1051,Şırnak,430006.0,3692,9923,9707,3908,7078,15,60.752472


In [5]:
# Linear Regression example for opened_inside_year:

# Drop non-feature columns from the DataFrame
X = data.drop(columns=['opened_inside_year','carried_over_from','finished','carried_over_to'])

# Separate the target variable
y = data['opened_inside_year']

# Split data into training and testing sets
# First 81 row is the year 2021
X_train, X_test, y_train, y_test = X.iloc[81:], X.iloc[:81], y.iloc[81:], y.iloc[:81]

# Preprocessing
# One-hot encode the city column
# Scale the population column
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), ['city']),
        ('pop_scaler', RobustScaler(), ['population']),
        ('density_scaler', RobustScaler(), ['population_density'])
    ]
)

# Define the model
model = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

# Train the model
model.fit(X_train, y_train)

# Predict on test set
y_pred = model.predict(X_test)

# Print predicted and real y-values for the first 10 rows
for i in range(10):
    print("Predicted:", y_pred[i], "\tReal:", y_test.iloc[i])

# Calculate absolute percentage error for each prediction
absolute_percentage_errors = np.abs((y_test - y_pred) / y_test)

# Calculate mean absolute percentage error
mape = np.mean(absolute_percentage_errors)

# Convert MAPE to accuracy (accuracy = 1 - MAPE)
accuracy = 1 - mape

# Convert accuracy to percentage
percentage_accuracy = accuracy * 100

print("Average Percentage Accuracy:", percentage_accuracy)

Predicted: 742599.0428753512 	Real: 841644
Predicted: 56942.4874887957 	Real: 67719
Predicted: 24002.339209752856 	Real: 25475
Predicted: 17081.80614832054 	Real: 18984
Predicted: 64681.81127395927 	Real: 73542
Predicted: 29381.437749284363 	Real: 33447
Predicted: 241254.11373865424 	Real: 305541
Predicted: 72443.31311257265 	Real: 87538
Predicted: 56220.25231353469 	Real: 59477
Predicted: 71668.69688991272 	Real: 83130
Average Percentage Accuracy: 84.88081242395148
